In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Read original dataset
full_data = pd.read_csv("CC_LCL-FullData.csv")

# Convert datetime column to pandas DateTime type
full_data['DateTime'] = pd.to_datetime(full_data['DateTime'])

# Extract the year
full_data['year'] = full_data['DateTime'].dt.year

# Only use year 2013
full_data = full_data[full_data['year'] == 2013]
full_data.drop(columns=['year'], inplace=True)

# Extract day of the year
full_data['dayoftheyear'] = full_data['DateTime'].dt.dayofyear

# Check if the day is a weekend (Saturday=5, Sunday=6)
full_data['is_weekend'] = full_data['DateTime'].dt.dayofweek >= 5

# Extract hour
full_data['hour'] = full_data['DateTime'].dt.hour

# Drop DateTime column as it is not needed anymore
full_data.drop(columns=["DateTime"], inplace=True)

# Select only standard customers 
full_data = full_data[full_data["stdorToU"] == "Std"]
full_data.drop(columns=["stdorToU"], inplace=True)

# Encode the ID of meters
label_encoder_lclid = LabelEncoder()
full_data['LCLid'] = label_encoder_lclid.fit_transform(full_data['LCLid'])
full_data['LCLid'] = full_data['LCLid'].astype('int16')

# Ensure that target does not contain NULLs and is numerric 
full_data['KWH/hh (per half hour) '] = pd.to_numeric(full_data['KWH/hh (per half hour) '], errors='coerce').fillna(0.0).astype('float16')

# Calculate hourly consumption 
hourly_data = full_data.groupby(['LCLid','dayoftheyear','hour','is_weekend'])['KWH/hh (per half hour) '].sum().reset_index()
hourly_data = hourly_data.rename(columns={'KWH/hh (per half hour) ': 'KWH/hh (per hour)'})

# Sort the data 
hourly_data.sort_values(by=['LCLid', 'dayoftheyear', 'hour'], inplace=True)

# Save the DataFrame to a CSV file
hourly_data.to_csv('Preprocessed_data.csv', index=False)

In [ ]:
hourly_data.head(100)

,LCLid,dayoftheyear,hour,is_weekend,KWH/hh (per hour)
0,0,1,0,False,0.459961
1,0,1,1,False,0.426025
2,0,1,2,False,0.411011
3,0,1,3,False,0.395020
4,0,1,4,False,0.221008
...,...,...,...,...,...
95,0,4,23,False,0.449951
96,0,5,0,True,0.402954
97,0,5,1,True,0.396973
98,0,5,2,True,0.395020


In [ ]:
print(hourly_data.isnull().sum())

LCLid                0
dayoftheyear         0
hour                 0
is_weekend           0
KWH/hh (per hour)    0
dtype: int64


In [ ]:
print(hourly_data.dtypes)

LCLid                  int16
dayoftheyear           int32
hour                   int32
is_weekend              bool
KWH/hh (per hour)    float32
dtype: object


In [ ]:


# Check the data types in the 'LCLid' column
print(hourly_data['LCLid'].apply(type).unique())

# Check the data types in the 'dayoftheyear' column
print(hourly_data['dayoftheyear'].apply(type).unique())

# Check the data types in the 'hour' column
print(hourly_data['hour'].apply(type).unique())

# Check the data types in the 'hour' column
print(hourly_data['is_weekend'].apply(type).unique())

# Check the data types in the 'hour' column
print(hourly_data['KWH/hh (per hour)'].apply(type).unique())

[<class 'int'>]
[<class 'int'>]
[<class 'int'>]
[<class 'bool'>]
[<class 'float'>]


In [ ]:
hourly_data.nunique()

In [ ]:
hourly_data['LCLid'].duplicated().sum()